# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core 
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
import logging


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-automl-timeseries'
experiment=Experiment(ws, experiment_name)
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
project_folder = './capstone-project-folder'


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

ml-azure-ws1
ml-azure-demo
eastus2
bb379b6e-1466-40b0-9fa1-cadacc0825e6


## Compute Target

In [4]:


from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "capstone-auto-ml-forecasting"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded..........................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


# Reading Dataset

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "timeseries_forecasting"
description_text = "Timeseries forecasting dataset for daily electricity production"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://mlazurews14714951657.blob.core.windows.net/azureml-blobstore-fd6fd0f9-e3a8-4250-a681-5da3738c7ef2/UI/2022-11-25_060252_UTC/Electric_Production.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,IPG2211A2N
count,397.000000
mean,88.847218
std,15.387834
min,55.315100
25%,77.105200
50%,89.779500
75%,100.524400
max,129.404800


In [6]:
dataset.take(5).to_pandas_dataframe()

,DATE,IPG2211A2N
0,1985-01-01,72.5052
1,1985-02-01,70.6720
2,1985-03-01,62.4502
3,1985-04-01,57.4714
4,1985-05-01,55.3151


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = ForecastingParameters(time_column_name='DATE', 
                                               forecast_horizon=12,
                                               freq='MS')

automl_config = AutoMLConfig(task='forecasting',
                            compute_target=compute_target,
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=15,
                             enable_early_stopping=True,
                             training_data=dataset,
                             label_column_name="IPG2211A2N",
                             n_cross_validations="auto", #" Could be customized as an integer
                             cv_step_size = "auto", # Could be customized as an integer
                             enable_ensembling=False,
                             debug_log = "automl_errors.log",
                             verbosity=logging.INFO,
                             forecasting_parameters=forecasting_parameters)                                               

In [ ]:
# from azureml.train.automl import AutoMLConfig

# automl_settings = {
#     "experiment_timeout_minutes": 20,
#     "max_concurrent_iterations": 4,
#     "primary_metric" : 'normalized_root_mean_squared_error',
#     "n_cross_validations": 5
# }
# automl_config = AutoMLConfig(compute_target=compute_target,
#                              task = "forecasting",
#                              training_data=dataset,
#                              time_column_name="DATE", 
#                              forecast_horizon = 10,
#                              label_column_name="IPG2211A2N",  
#                              path = project_folder,
#                              enable_early_stopping= True,
#                              #featurization= 'auto',
#                              debug_log = "automl_errors.log",
#                              **automl_settings
#                             )

In [8]:
# Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on capstone-auto-ml-forecasting with default configuration
Running on remote compute: capstone-auto-ml-forecasting


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-automl-timeseries,AutoML_9920f1bc-38e8-4fe9-ac68-d7b2df464fb7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Time Series ID detection
STATUS:       PASSED
DESCRIPTION:  The data set was analyzed, and no duplicate time index were detected.
              Learn more about time-series forecasting configurations: https://aka.ms/AutomatedMLForecastingConfiguration

********************************************************************************************

TYPE:         Short series handling
STATUS:       PASSED
DESCRIPTION:  Automated ML detected enough data points for each series in the input data to continue with training.
              Learn more about short series handling: https://aka.ms/AutomatedMLShortSeriesHandling

********************************************************************************************

TYPE:         Frequency detection
STATUS:       PASSED
DESCRIPTION:  The time series was analyzed,

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_9920f1bc-38e8-4fe9-ac68-d7b2df464fb7',
 'target': 'capstone-auto-ml-forecasting',
 'status': 'Completed',
 'startTimeUtc': '2022-11-28T09:45:31.226399Z',
 'endTimeUtc': '2022-11-28T10:03:05.292587Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': 'auto',
  'target': 'capstone-auto-ml-forecasting',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-automl-timeseries","subscription_id":"bb379b6e-1466-40b0-9fa1-cadacc0825e6","resource_group":"ml-azure-demo","workspace_name":"ml-azure-ws1","re

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
# Retrieve and save your best automl model.
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

Package:azureml-automl-runtime, training version:1.46.1, current version:1.44.0
Package:azureml-core, training version:1.46.0, current version:1.44.0
Package:azureml-dataprep, training version:4.5.7, current version:4.2.2
Package:azureml-dataprep-rslex, training version:2.11.4, current version:2.8.1
Package:azureml-dataset-runtime, training version:1.46.0, current version:1.44.0
Package:azureml-defaults, training version:1.46.0, current version:1.44.0
Package:azureml-interpret, training version:1.46.0, current version:1.44.0
Package:azureml-mlflow, training version:1.46.0, current version:1.44.0
Package:azureml-pipeline-core, training version:1.46.0, current version:1.44.0
Package:azureml-responsibleai, training version:1.46.0, current version:1.44.0
Package:azureml-telemetry, training version:1.46.0, current version:1.44.0
Package:azureml-train-automl-client, training version:1.46.0, current version:1.44.0
Package:azureml-train-automl-runtime, training version:1.46.1, current version:

In [11]:
best_run_metrics

{'median_absolute_error': 1.9004204578754582,
 'mean_absolute_percentage_error': 2.4951145835101864,
 'r2_score': 0.8588025737354241,
 'mean_absolute_error': 2.633605929487178,
 'normalized_root_mean_squared_error': 0.04894180999217927,
 'normalized_root_mean_squared_log_error': 0.03993326799460134,
 'explained_variance': 0.8714498520972767,
 'normalized_median_absolute_error': 0.025650265257862542,
 'root_mean_squared_error': 3.626084019777564,
 'normalized_mean_absolute_error': 0.03554618158107238,
 'spearman_correlation': 0.9464112587251059,
 'root_mean_squared_log_error': 0.03353119655571562,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_9920f1bc-38e8-4fe9-ac68-d7b2df464fb7_23/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_9920f1bc-38e8-4fe9-ac68-d7b2df464fb7_23/predicted_true',
 'forecast_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_9920f1bc-38e8-4fe9-ac68-d7b2df464fb7_23/forecast_table'}

In [15]:
import joblib
joblib.dump(fitted_model, 'model.pkl')

['model.pkl']

In [13]:
from azureml.core import Model
model = Model.register(
    workspace=ws, 
    model_name='forecasting_automl_model', 
    model_path='./model.pkl'
)

Registering model forecasting_automl_model


In [14]:
best_run.get_details()

{'runId': 'AutoML_9920f1bc-38e8-4fe9-ac68-d7b2df464fb7_23',
 'target': 'capstone-auto-ml-forecasting',
 'status': 'Completed',
 'startTimeUtc': '2022-11-28T09:57:19.797555Z',
 'endTimeUtc': '2022-11-28T09:57:42.468832Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '3840ebb82b060782c0c1aaabbe352751ed0cb999',
  'pipeline_spec': '{"objects": [{"class_name": "MinMaxScaler", "module": "sklearn.preprocessing", "param_args": [], "param_kwargs": {}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "DecisionTreeRegressor", "module": "sklearn.tree", "param_args": [], "param_kwargs": {"criterion": "friedman_mse", "max_features": null, "min_samples_leaf": 0.029530863384643957, "min_samples_split": 0.000630957344480193, "splitter": "best"}, "prepared_kwargs": {}, "spec_class": "sklearn"}], "pipeline_id": "3840ebb82b060782c0c1aaabbe352751ed0cb999", "module": "sklearn.pipeline", "class_name": "Pipeline", "pipeline_name": "{ MinMaxScaler, DecisionTre

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
script_file_name = 'score_fcast.py'
best_run.download_file("outputs/scoring_file_v_1_0_0.py",script_file_name)

In [17]:
import os
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/demo-ci-udacity/code/Users/sgoel.azure.1509/starter_file'

# Create Environment

In [18]:
from azureml.core import Environment
# Get the environment
from azureml.automl.core.shared import constants

best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'conda_dependencies.yml')
service_env = Environment.from_conda_specification(name='deployment-env', file_path='conda_dependencies.yml')


TODO: In the cell below, send a request to the web service you deployed to test it.

# Creating Inference Configuration

In [19]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script='score_fcast.py',
                                              source_directory= ".",
                                              environment=service_env)

In [20]:
from azureml.core.webservice import AciWebservice

deploy_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=2, auth_enabled=True, enable_app_insights=True
)

In [21]:
from azureml.core.model import Model
aci_service_name = "automl-forecast"
service = Model.deploy(
    ws,
    aci_service_name,
    [model],
    inference_config,
    deploy_config,
    overwrite=True
   
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-11-28 10:11:00+00:00 Creating Container Registry if not exists.
2022-11-28 10:11:00+00:00 Registering the environment.
2022-11-28 10:11:00+00:00 Use the existing image.
2022-11-28 10:11:00+00:00 Generating deployment configuration.
2022-11-28 10:11:01+00:00 Submitting deployment to compute.
2022-11-28 10:11:04+00:00 Checking the status of deployment automl-forecast..
2022-11-28 10:13:28+00:00 Checking the status of inference endpoint automl-forecast.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [27]:
import requests
import json
from azureml.core import Webservice
import pandas as pd
service = Webservice(workspace=ws, name= "automl-forecast")
scoring_uri = service.scoring_uri
print(scoring_uri)
# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"
# Make the request and display the response and logs
#data = pd.DataFrame({"DATE": pd.Series(["2/1/2018"])})
data = {
  "data": [
    {
      "DATE": "2018-02-01T00:00:00.000Z"

    }, {
      "DATE": "2018-03-01T00:00:00.000Z"

    },
     {
      "DATE": "2018-04-01T00:00:00.000Z"

    }

  ],
  "quantiles": [
    0.025,
    0.975
  ]
}
input_data = json.dumps(data)
# print(type(input_data))
resp = requests.post(scoring_uri, data=input_data, headers=headers)
print(resp.json())

http://588b8cb2-c200-4d6d-b9cf-5bba0504a1be.eastus2.azurecontainer.io/score
{"forecast": [113.1948, 97.3306894736842, 89.12116666666667], "prediction_interval": ["[106.14687699138926, 120.24272300861074]", "[87.3634211683455, 107.2979577790229]", "[76.91380592791916, 101.32852740541416]"], "index": [{"DATE": 1517443200000}, {"DATE": 1519862400000}, {"DATE": 1522540800000}]}


In [28]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_2e749d748707b6dae1f9e16287d17b3d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_2e749d748707b6dae1f9e16287d17b3d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_2e749d748707b6dae1f9e16287d17b3d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_2e749d748707b6dae1f9e16287d17b3d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2022-11-28T10:13:16,635263200+00:00 - iot-server/run 
2022-11-28T10:13:16,635931200+00:00 - rsyslog/run 
bash: /azureml-envs/azureml_2e749d748707b6dae1f9e16287d17b3d/lib/libtinfo.so.6: no version information available (required by bash)
2022-11-28T10:13:16,649030700+00:00 - gunicorn/run 
2022-11-28T10:13:16,654711800+00:00 | gunicorn/run | 
2022-11-28T10:13:16,661405400+00:00 | gunicorn/run | ####################################

# Delete the web service

In [30]:
service.delete()

# Shutdown the computes

In [31]:
try:
    instance = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)

    instance.delete()
    instance.wait_for_completion(show_output=True)
    print('Deleted compute resource')

except ComputeTargetException as e:
    print('Already deleted!')

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Already deleted!


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
